In [39]:
import tensorflow as tf
from skimage import io
import matplotlib.pyplot as plt
import random
import os
import cv2
import numpy as np
from tqdm import tqdm 

def image_to_patches(image, patch_height, patch_width):
    # resize image so that it's dimensions are dividable by patch_height and patch_width
    image_height = tf.cast(tf.shape(image)[0], dtype=tf.float32)
    image_width = tf.cast(tf.shape(image)[1], dtype=tf.float32)
    height = tf.cast((tf.ceil(image_height / patch_height)-1) * patch_height, dtype=tf.int32)
    width = tf.cast((tf.ceil(image_width / patch_width) - 1) * patch_width, dtype=tf.int32)

    num_rows = height // patch_height
    num_cols = width // patch_width
    # make zero-padding
    image = tf.squeeze(tf.image.resize_image_with_crop_or_pad(image, height, width))

    # get slices along the 0-th axis
    image = tf.reshape(image, [num_rows, patch_height, width, -1])
    # h/patch_h, w, patch_h, c
    image = tf.transpose(image, [0, 2, 1, 3])
    # get slices along the 1-st axis
    # h/patch_h, w/patch_w, patch_w,patch_h, c
    image = tf.reshape(image, [num_rows, num_cols, patch_width, patch_height, -1])
    # num_patches, patch_w, patch_h, c
    image = tf.reshape(image, [num_rows * num_cols, patch_width, patch_height, -1])
    # num_patches, patch_h, patch_w, c
    return tf.transpose(image, [0, 2, 1, 3])

def get_random_n_patches(patches, k):
    n = patches.shape[0]
    if n < 2:
        return patches
    while(True):
        if(k <= n):
            rand = random.sample(range(n), k)
            break
        else:
            k -= 3
    return patches[rand]

def patch_img_to_file(base_path, file_name, dest, label_anno):
    global data
    global label
    image = io.imread(os.path.join(base_path, file_name))
    #print(file_name)
    tile_size = 355 #512
    image = tf.constant(image)
    tiles = image_to_patches(image, tile_size, tile_size)
    sess = tf.Session()
    I, tiles = sess.run([image, tiles])
#     print(I.shape)  #anh goc
#     print(tiles.shape)
    tiles = get_random_n_patches(tiles, 5)
    j = 1
    for i, tile in enumerate(tiles):
        tmp = os.path.splitext(file_name)
        new_name = tmp[0] + "_patch" + str(j) + tmp[1]
        cv2.imwrite(os.path.join(dest, new_name), tile)
        j+=1
        data.append(tile)
        label.append(label_anno)

BASE1 = "/root/patchcore-inspection/pantos_data/reuse_img/train/real/"
DES1 = "/root/laplacian_classify_recapture_image/dataset/train/real/"
BASE2 = "/root/patchcore-inspection/pantos_data/reuse_img/train/fake/"
DES2 = "/root/laplacian_classify_recapture_image/dataset/train/fake/"
LOG0 = "/root/laplacian_classify_recapture_image/logs/1"
LOG1 = "/root/laplacian_classify_recapture_image/logs/2"



In [42]:
os.makedirs(DES1, exist_ok=True)
os.makedirs(DES2, exist_ok=True)
os.makedirs(LOG0, exist_ok=True)
os.makedirs(LOG1, exist_ok=True)


In [37]:
for item in os.listdir(BASE1):
    patch_height = patch_width = 355
    try:
        image_ori = io.imread(os.path.join(BASE1, item))
        
        image = tf.constant(image_ori)
        image_height = tf.cast(tf.shape(image)[0], dtype=tf.float32)
        image_width = tf.cast(tf.shape(image)[1], dtype=tf.float32)
        height = tf.cast((tf.ceil(image_height / patch_height)-1) * patch_height, dtype=tf.int32)
        width = tf.cast((tf.ceil(image_width / patch_width) - 1) * patch_width, dtype=tf.int32)

        num_rows = height // patch_height
        num_cols = width // patch_width
        # make zero-padding
        image = tf.image.resize_image_with_crop_or_pad(image, height, width)
        image = tf.squeeze(image)

        # print('PASS', item, image_ori.shape)
    except:
        # print(item)?
        print('FAILED', item, image_ori.shape)

FAILED 1089613272-1.1.1@1@2@6648658@206907747.jpg (360, 800, 3)


In [ ]:

image = tf.constant(image_ori)
image_height = tf.cast(tf.shape(image)[0], dtype=tf.float32)
image_width = tf.cast(tf.shape(image)[1], dtype=tf.float32)
height = tf.cast((tf.ceil(image_height / patch_height)-1) * patch_height, dtype=tf.int32)
width = tf.cast((tf.ceil(image_width / patch_width) - 1) * patch_width, dtype=tf.int32)

num_rows = height // patch_height
num_cols = width // patch_width
# make zero-padding
image = tf.image.resize_image_with_crop_or_pad(image, height, width)
image = tf.squeeze(image)

In [43]:

data = []
label = []
cnt1 = 1
cnt2 = 1
total = 1
for item in tqdm(os.listdir(BASE1)):
    patch_img_to_file(BASE1, item, DES1, 0)
    # if(cnt1 % 100 == 0):
    np_data = np.asarray(data)
    np_label = np.asarray(label)
    name_data = "data_natural_" + str(cnt1)
    name_label = "label_natural_" + str(cnt1)
    np.save(os.path.join(LOG0, name_data), np_data)
    np.save(os.path.join(LOG0, name_label), np_label)
    data = []
    label = []
    print("saved at number " + str(cnt1) + "--natural images--")
    cnt1+=1
    total +=1
print("******************DONE-phase 1******************************")

data = []
label = []
for item in tqdm(os.listdir(BASE2)):
    patch_img_to_file(BASE2, item, DES2, 1)
    # if(cnt2 % 75 == 0):
    np_data = np.asarray(data)
    np_label = np.asarray(label)
    name_data = "data_recapture_" + str(cnt2)
    name_label = "label_recapture_" + str(cnt2)
    np.save(os.path.join(LOG1, name_data), np_data)
    np.save(os.path.join(LOG1, name_label), np_label)
    data = []
    label = []
    print("saved at number " + str(cnt2) + "--recapture images--")
    cnt2+=1
    total +=1
print("total_image: " + str(total))
# data = np.asarray(data)
# label = np.asarray(label)
# print(data.shape)
# print(label.shape)
# np.save("data", data)
# np.save("label", label)

  0%|          | 0/100 [00:00<?, ?it/s]2023-10-18 04:47:44.798574: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-10-18 04:47:44.798604: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
  1%|          | 1/100 [00:04<08:11,  4.97s/it]

saved at number 1--natural images--


2023-10-18 04:47:49.745558: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-10-18 04:47:49.745584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
  2%|▏         | 2/100 [00:09<07:18,  4.48s/it]

saved at number 2--natural images--


2023-10-18 04:47:53.887053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-10-18 04:47:53.887080: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
  3%|▎         | 3/100 [00:13<06:57,  4.31s/it]

...

2023-10-18 04:56:06.311274: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-10-18 04:56:06.311299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
 12%|█▏        | 7/58 [00:34<04:10,  4.91s/it]

saved at number 7--recapture images--


2023-10-18 04:56:11.228244: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1181] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-10-18 04:56:11.228272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1187]      
 14%|█▍        | 8/58 [00:39<04:03,  4.88s/it]

saved at number 8--recapture images--


ValueError: all dims of 'image.shape' must be > 0: (1, 710, 0, 3)

In [ ]:

data = []
label = []
for item in tqdm(os.listdir(BASE2)):
    patch_img_to_file(BASE2, item, DES2, 1)
    # if(cnt2 % 75 == 0):
    np_data = np.asarray(data)
    np_label = np.asarray(label)
    name_data = "data_recapture_" + str(cnt2)
    name_label = "label_recapture_" + str(cnt2)
    np.save(os.path.join(LOG1, name_data), np_data)
    np.save(os.path.join(LOG1, name_label), np_label)
    data = []
    label = []
    print("saved at number " + str(cnt2) + "--recapture images--")
    cnt2+=1
    total +=1
print("total_image: " + str(total))
# data = np.asarray(data)
# label = np.asarray(label)
# print(data.shape)
# print(label.shape)
# np.save("data", data)
# np.save("label", label)